In [1]:
%cd /scratch/edk202/ngram-prep

%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8

%pip install -e . --no-build-isolation -q

/scratch/edk202/ngram-prep
env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Auto-reload packages
%load_ext autoreload
%autoreload 2

In [3]:
# Standard stuff
from pathlib import Path

# NLTK stuff
from nltk.corpus import stopwords; stopwords = set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer; lemmatizer = WordNetLemmatizer()

# Raw n-gram acquisition stuff
from ngram_acquire.pipeline.orchestrate import download_and_ingest_to_rocksdb
from ngram_acquire.pipeline.logger import setup_logger

# Cython utilities
from ngram_filter.config import PipelineConfig, FilterConfig
from ngram_filter.pipeline.orchestrator import build_processed_db

In [4]:
setup_logger(
    db_path="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

PosixPath('/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db/ngram_download_20250914_205048.log')

# **Download 5-Grams and Ingest to a RocksDB Database**

In [ ]:
download_and_ingest_to_rocksdb(
    ngram_size = 5,
    repo_release_id = "20200217",
    repo_corpus_id = "eng",
    db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db",
    file_range = (0, 19422),
    random_seed = 21,
    workers = 39,
    use_threads = False,
    ngram_type = "tagged",
    overwrite = False,
    write_batch_size = 100_000,
    open_type = "write:packed24",
    post_compact = False
)

Start Time: 2025-09-06 21:58:02

Download & Ingestion Configuration
Ngram repository:           https://storage.googleapis.com/books/ngrams/books/20200217/eng/eng-5-ngrams_exports.html
RocksDB database path:      /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
File index range:           0 to 19422 (count ~ 19423)
Total files available:      19423
Files to process:           9657
First file URL:             http://storage.googleapis.com/books/ngrams/books/20200217/eng/5-16099-of-19423.gz
Last file URL:              http://storage.googleapis.com/books/ngrams/books/20200217/eng/5-03676-of-19423.gz
Ngram size:                 5
Ngram filtering:            tagged
Overwrite mode:             False
Files to skip (processed):  9766
Write batch size:           100,000
Worker processes/threads:   39 (processes)



Processing Files:   4%|▍         | 411/9657 [10:44<2:19:25,  1.11files/s] 

## Process the Ngrams

In [ ]:
src_db = Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db")
dst_db = src_db.parent / "5grams_processed.db"
tmp_dir = src_db.parent / "processing_tmp"
wht_path ="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams_processed.db/whitelist.txt"

# Default configs or override as desired
pipeline_config = PipelineConfig(
    src_db=src_db,
    dst_db=dst_db,
    tmp_dir=tmp_dir,
    readers=8,
    force_restart=False,
    progress_every_s=60.0,
)

filter_config = FilterConfig(
    stop_set=stopwords,
    lemma_gen=lemmatizer,
    whitelist_path=wht_path
)

build_processed_db(pipeline_config, filter_config)

N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Configuration:
════════════════════════════════════════════════════════════════════════════════════════════════════
  Workers: 8
  Work units: 64
  Source: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
  Destination: 02/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
  Buffer: 25,000 items, 16MB
  Profile: write:packed24
  Input whitelist: .../Google_Books/20200217/eng/5gram_files/1grams_processed.db/whitelist.txt
    All tokens (min count: 1)
  Output whitelist: None
  Loading whitelist...
  Loaded 60,000 tokens

Phase 1: Creating work units...
════════════════════════════════════════════════════════════════════════════════════════════════════
  Force restart requested - clearing existing work units
  Creating 64 work units using ASCII range...
  Created 64 work units covering range 0x21-0x7e
  Created 64 work uni